# Breast Cancer Diagnosis Using k-Nearest Neighbors

## Project Overview

This mini-project illustrates the process of analyzing the Breast Cancer Wisconsin (Diagnostic) dataset using Python, pandas, and scikit-learn. We'll develop a k-Nearest Neighbors (kNN) classifier to assist in diagnosing breast cancer. The analysis includes loading and transforming the data, generating descriptive statistics, training the model, and evaluating its performance.

### 1. Load and Explore Dataset

First, we load the Breast Cancer Wisconsin dataset from scikit-learn and explore its attributes.

In [4]:
import warnings
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer

warnings.filterwarnings('ignore')
np.set_printoptions(precision=3)

# Load the Breast Cancer Wisconsin dataset
cancer = load_breast_cancer()

# Display dataset keys
print(cancer.keys())

# Display dataset description
print(cancer.DESCR)

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])
.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

:Number of Instances: 569

:Number of Attributes: 30 numeric, predictive attributes and the class

:Attribute Information:
    - radius (mean of distances from center to points on the perimeter)
    - texture (standard deviation of gray-scale values)
    - perimeter
    - area
    - smoothness (local variation in radius lengths)
    - compactness (perimeter^2 / area - 1.0)
    - concavity (severity of concave portions of the contour)
    - concave points (number of concave portions of the contour)
    - symmetry
    - fractal dimension ("coastline approximation" - 1)

    The mean, standard error, and "worst" or largest (mean of the three
    worst/largest values) of these features were computed for each image,
    resulting in 30 f

### 2. Data Transformation

We transform the dataset into a pandas DataFrame for easier manipulation.

In [5]:
# Function to transform the dataset into a pandas DataFrame
def make_cancer_dataframe():
    cancer_df = pd.DataFrame(cancer['data'], columns=cancer['feature_names'])
    cancer_df['target'] = cancer['target']
    return cancer_df

# Create DataFrame
cancer_df = make_cancer_dataframe()
print(f"DataFrame shape: {cancer_df.shape}")
cancer_df.head()

DataFrame shape: (569, 31)


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


### 3. Descriptive Statistics

Calculate and display the class distribution of the dataset.

In [6]:
# Function to calculate class distribution
def get_target_distro(df):
    malignant = (df['target'] == 0).sum()
    benign = (df['target'] == 1).sum()
    return pd.Series([malignant, benign], index=['malignant', 'benign'])

# Display class distribution
target_distro = get_target_distro(cancer_df)
print(target_distro)

malignant    212
benign       357
dtype: int64


### 4. Data Preparation for Model

Prepare the features (X) and target (y) for model training.

In [7]:
# Function to prepare X (features) and y (target)
def prepare_X_y(df):
    X = df.iloc[:, :-1]
    y = df['target']
    return X, y

# Prepare the data
X, y = prepare_X_y(cancer_df)
print(f"Features shape: {X.shape}, Target shape: {y.shape}")

Features shape: (569, 30), Target shape: (569,)


### 5. Model Training and Evaluation

#### Train-Test Split

Split the dataset into training and testing sets for model evaluation.

In [8]:
from sklearn.model_selection import train_test_split

# Function to split the data into training and testing sets
def get_train_test(X, y, random_state=42):
    return train_test_split(X, y, random_state=random_state)

# Perform train-test split
X_train, X_test, y_train, y_test = get_train_test(X, y)
print(f"Train shapes: {X_train.shape, y_train.shape}, Test shapes: {X_test.shape, y_test.shape}")

Train shapes: ((426, 30), (426,)), Test shapes: ((143, 30), (143,))


#### Train kNN Classifier

Train a kNN classifier with k=1.

In [9]:
from sklearn.neighbors import KNeighborsClassifier

# Function to train a kNN classifier
def k_nearest_neighbors(X_train, y_train, n_neighbors=1):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    return knn

# Train kNN classifier
knn = k_nearest_neighbors(X_train, y_train)
print(knn)

KNeighborsClassifier(n_neighbors=1)


#### Evaluate the Model

Predict the class label for the mean vector of the training data and evaluate the model on the test set.

In [10]:
# Function to predict class label for the mean vector
def knn_predict_on_mean(knn, X_train):
    means = X_train.mean().values.reshape(1, -1)
    return knn.predict(means)

# Predict on mean vector
mean_prediction = knn_predict_on_mean(knn, X_train)
print(f"Prediction on mean vector: {mean_prediction}")

# Function to predict class labels on the test set
def knn_predict_on_test(knn, X_test):
    return knn.predict(X_test)

# Predict on test set
test_predictions = knn_predict_on_test(knn, X_test)
print(test_predictions)

# Function to compute model accuracy
def knn_score_prediction(knn, X_test, y_test):
    return knn.score(X_test, y_test)

# Evaluate model accuracy
accuracy = knn_score_prediction(knn, X_test, y_test)
print(f"Model accuracy: {accuracy}")

Prediction on mean vector: [1]
[1 0 0 1 1 0 0 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0
 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 0 0 1 1 1 0 0 1 1 0 0 1 1
 1 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 0 1 1 1 0 1 1 0
 1 0 0 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1 0 1]
Model accuracy: 0.9300699300699301


### 6. Hyper-Parameter Tuning

Perform a parameter sweep for odd values of k from 1 to 19 and find the optimal k.

In [11]:
# Function to perform hyper-parameter tuning
def knn_hyperparameter_tuning(X_train, y_train, X_test, y_test):
    best_accuracy = 0
    k_best = None
    for k in range(1, 20, 2):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        accuracy = knn.score(X_test, y_test)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            k_best = k
    return k_best

# Perform hyper-parameter tuning
best_k = knn_hyperparameter_tuning(X_train, y_train, X_test, y_test)
print(f"Optimal k: {best_k}")

Optimal k: 11


### 7. One-Hot Encoding Example

Example of using one-hot encoding on a secondary dataset to illustrate handling categorical data.

In [16]:
# Load a sample housing prices dataset for illustration
def get_house_prices_data():
    data = pd.read_csv(r'C:/Users/akama/Downloads/housing_prices.csv')
    X = data.drop("SalePrice", axis=1)
    y = data["SalePrice"]
    X["Electrical"].fillna("Mix", inplace=True)
    return X, y

# Function to split the housing prices data
def get_house_prices_split(random_state=42):
    X, y = get_house_prices_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state)
    return X_train, X_test, y_train, y_test

# Function to perform one-hot encoding
def one_hot_encoding(encode_features, handle_unk="error"):
    X_train, X_test, y_train, y_test = get_house_prices_split()
    encoder = OneHotEncoder(handle_unknown=handle_unk, sparse_output=False, dtype=int)
    ohe_indices = [X_train.columns.get_loc(feature) for feature in encode_features]
    encoder.fit(X_train.iloc[:, ohe_indices])
    ohe_features_array = encoder.transform(X_train.iloc[:, ohe_indices])
    ohe_feature_names = encoder.get_feature_names_out(input_features=encode_features)
    ohe_df_train = pd.DataFrame(ohe_features_array, columns=ohe_feature_names, index=X_train.index)
    X_train = pd.concat([X_train, ohe_df_train], axis=1)
    ohe_features_array_test = encoder.transform(X_test.iloc[:, ohe_indices])
    ohe_df_test = pd.DataFrame(ohe_features_array_test, columns=ohe_feature_names, index=X_test.index)
    X_test = pd.concat([X_test, ohe_df_test], axis=1)
    return X_train, X_test, y_train, y_test, ohe_feature_names

# Test one-hot encoding function
X_tr, X_te, y_tr, y_te, ohe_feat = one_hot_encoding(["BldgType"])
print(ohe_feat)
display(X_tr.head())

['BldgType_1Fam' 'BldgType_2fmCon' 'BldgType_Duplex' 'BldgType_Twnhs'
 'BldgType_TwnhsE']


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscVal,MoSold,YrSold,SaleType,SaleCondition,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE
1023,1024.0,120.0,RL,43.0,3182.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,5.0,2008.0,WD,Normal,0,0,0,0,1
810,811.0,20.0,RL,78.0,10140.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,1.0,2006.0,WD,Normal,1,0,0,0,0
1384,1385.0,50.0,RL,60.0,9060.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,10.0,2009.0,WD,Normal,1,0,0,0,0
626,627.0,20.0,RL,NaN,12342.0,Pave,NaN,IR1,Lvl,AllPub,...,600.0,8.0,2007.0,WD,Normal,1,0,0,0,0
813,814.0,20.0,RL,75.0,9750.0,Pave,NaN,Reg,Lvl,AllPub,...,500.0,4.0,2007.0,COD,Normal,1,0,0,0,0


## Summary

This brief project demonstrates the process of creating a kNN classifier for diagnosing breast cancer using the Breast Cancer Wisconsin (Diagnostic) dataset. It includes data transformation, model training, evaluation, and hyper-parameter tuning. An additional one-hot encoding example illustrates handling categorical data. For further insights and projects, please check out my GitHub portfolio.